In [60]:
import pandas as pd
import numpy as np
import os
import sqlalchemy
import sqlite3 as sql
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import pickle
import joblib

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler


database = "../KAGGLE_PRUBAS_DATA/DATA/diamonds_train.db"
connection = sql.connect(database)

query = '''SELECT tr.index_id, c.clarity, cl.color, ct.cut, dm.depth, dm.'table', dm.x, dm.y, dm.z, tr.price, ci.city, tr.carat
FROM diamonds_properties pr
   INNER JOIN diamonds_clarity c ON pr.clarity_id = c.clarity_id
   INNER JOIN diamonds_color cl on pr.color_id = cl.color_id
   INNER JOIN diamonds_cut ct on pr.cut_id = ct.cut_id
   INNER JOIN diamonds_dimensions dm on pr.index_id = dm.index_id
   INNER JOIN diamonds_transactional tr on pr.index_id = tr.index_id
   INNER JOIN diamonds_city ci on tr.city_id = ci.city_id'''

diamonds = pd.read_sql_query(query,connection)





In [61]:
diamonds

,index_id,clarity,color,cut,depth,table,x,y,z,price,city,carat
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,VS2,J,Premium,62.4,58.0,6.83,6.79,4.25,4268,Dubai,1.21
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,VS2,H,Very Good,63.0,57.0,4.35,4.38,2.75,505,Kimberly,0.32
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,VS1,G,Fair,65.5,55.0,5.62,5.53,3.65,2686,Las Vegas,0.71
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,SI1,D,Good,63.8,56.0,4.68,4.72,3.00,738,Kimberly,0.41
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,SI1,G,Ideal,60.5,59.0,6.55,6.51,3.95,4882,Dubai,1.02
...,...,...,...,...,...,...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,VS1,G,Ideal,62.7,57.0,7.10,7.04,4.43,10070,Antwerp,1.34
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,SI2,F,Good,57.1,60.0,8.31,8.25,4.73,12615,Madrid,2.02
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,SI1,H,Ideal,62.7,56.0,6.37,6.42,4.01,5457,Kimberly,1.01
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,VS1,J,Ideal,61.9,54.3,4.45,4.47,2.76,456,Kimberly,0.33


ELIMINAMOS LOS OUTLIERS DE TODAS LAS COLUMNAS 

In [62]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.05)
    q3 = df_in[col_name].quantile(0.95)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [63]:
diamonds =remove_outlier(diamonds,"depth")
diamonds =remove_outlier(diamonds,"table")
diamonds =remove_outlier(diamonds,"x")
diamonds=remove_outlier(diamonds,"y")
diamonds =remove_outlier(diamonds,"z")
diamonds=remove_outlier(diamonds,"carat")

In [64]:
diamonds.describe()

,depth,table,x,y,z,price,carat
count,40426.000000,40426.000000,40426.000000,40426.000000,40426.000000,40426.000000,40426.000000
mean,61.750918,57.443311,5.728657,5.730911,3.536448,3926.733439,0.797180
std,1.408720,2.215827,1.123584,1.115002,0.696046,3990.022788,0.474116
min,52.700000,44.000000,0.000000,0.000000,0.000000,326.000000,0.200000
25%,61.000000,56.000000,4.710000,4.720000,2.910000,945.000000,0.400000
50%,61.800000,57.000000,5.690000,5.710000,3.520000,2396.000000,0.700000
75%,62.500000,59.000000,6.540000,6.540000,4.030000,5330.750000,1.040000
max,70.500000,70.000000,9.660000,9.630000,6.270000,18823.000000,3.510000


CREAMOS LAS VARIABLES DUMMIES DEL DATASET YA LIMPIO DE OUTLIERS

In [65]:

dummy_clarity = pd.get_dummies(diamonds["clarity"], prefix= "clarity")
dummy_color = pd.get_dummies(diamonds["color"], prefix= "color")
dummy_cut = pd.get_dummies(diamonds["cut"], prefix= "cut")
dummy_city = pd.get_dummies(diamonds["city"], prefix= "city")


diamonds_dummies_cleaned = pd.concat([diamonds, dummy_clarity,dummy_color,dummy_cut,dummy_city], axis=1)
lista_columnas = diamonds_dummies_cleaned.columns.values.tolist()

diamonds_dummies_cleaned

,index_id,clarity,color,cut,depth,table,x,y,z,price,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,VS2,J,Premium,62.4,58.0,6.83,6.79,4.25,4268,...,0,0,0,0,0,0,0,0,0,0
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,VS2,H,Very Good,63.0,57.0,4.35,4.38,2.75,505,...,1,0,0,0,0,0,0,0,0,0
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,VS1,G,Fair,65.5,55.0,5.62,5.53,3.65,2686,...,0,1,0,0,0,0,0,0,0,0
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,SI1,D,Good,63.8,56.0,4.68,4.72,3.00,738,...,1,0,0,0,0,0,0,0,0,0
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,SI1,G,Ideal,60.5,59.0,6.55,6.51,3.95,4882,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,f0bc79169405ebeb24e308055156b946ffd819db9b4f75...,VS1,G,Ideal,62.7,57.0,7.10,7.04,4.43,10070,...,0,0,0,0,0,0,0,0,0,0
40451,339916a23bf22b052b54cb2a9b36ee8418c1c68b46acad...,SI2,F,Good,57.1,60.0,8.31,8.25,4.73,12615,...,0,0,0,0,1,0,0,0,0,0
40452,46957922b99954654c1deb8d854c3f069bf118b2ce9415...,SI1,H,Ideal,62.7,56.0,6.37,6.42,4.01,5457,...,1,0,0,0,0,0,0,0,0,0
40453,9d733392d362d5c6f1d9b9659b601c7d4b5a1c1c8df579...,VS1,J,Ideal,61.9,54.3,4.45,4.47,2.76,456,...,1,0,0,0,0,0,0,0,0,0


ELIMINAMOS LAS FEATURES DE LAS CUALES HEMOS SACADO LAS DUMMIES

In [66]:
diamonds_dummies_cleaned = diamonds_dummies_cleaned.drop(['clarity','color','cut','city'], axis = 1)

HACEMOS UN LISTADO CON NUESTRAS FEATURES

In [67]:
features = ['depth',
 'table',
 'x',
 'y',
 'z',
 'carat',
 'clarity_I1',
 'clarity_IF',
 'clarity_SI1',
 'clarity_SI2',
 'clarity_VS1',
 'clarity_VS2',
 'clarity_VVS1',
 'clarity_VVS2',
 'color_D',
 'color_E',
 'color_F',
 'color_G',
 'color_H',
 'color_I',
 'color_J',
 'cut_Fair',
 'cut_Good',
 'cut_Ideal',
 'cut_Premium',
 'cut_Very Good',
 'city_Amsterdam',
 'city_Antwerp',
 'city_Dubai',
 'city_Kimberly',
 'city_Las Vegas',
 'city_London',
 'city_Luxembourg',
 'city_Madrid',
 'city_New York City',
 'city_Paris',
 'city_Surat',
 'city_Tel Aviv',
 'city_Zurich']

COMO VEMOS EN EL DATAFRAME DE DIAMONDS_DUMMIES_CLEANED TENEMOS UNA COLUMNA INDEX_ID QUE TENEMOS QUE ELIMINAR

In [68]:
diamonds_dummies_cleaned.head()

,index_id,depth,table,x,y,z,price,carat,clarity_I1,clarity_IF,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
0,5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91...,62.4,58.0,6.83,6.79,4.25,4268,1.21,0,0,...,0,0,0,0,0,0,0,0,0,0
1,6b86b273ff34fce19d6b804eff5a3f5747ada4eaa22f1d...,63.0,57.0,4.35,4.38,2.75,505,0.32,0,0,...,1,0,0,0,0,0,0,0,0,0
2,d4735e3a265e16eee03f59718b9b5d03019c07d8b6c51f...,65.5,55.0,5.62,5.53,3.65,2686,0.71,0,0,...,0,1,0,0,0,0,0,0,0,0
3,4e07408562bedb8b60ce05c1decfe3ad16b72230967de0...,63.8,56.0,4.68,4.72,3.00,738,0.41,0,0,...,1,0,0,0,0,0,0,0,0,0
4,4b227777d4dd1fc61c6f884f48641d02b4d121d3fd328c...,60.5,59.0,6.55,6.51,3.95,4882,1.02,0,0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
diamonds_dummies_cleaned = diamonds_dummies_cleaned.drop(columns = ["index_id"])

In [70]:
diamonds_dummies_cleaned

,depth,table,x,y,z,price,carat,clarity_I1,clarity_IF,clarity_SI1,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
0,62.4,58.0,6.83,6.79,4.25,4268,1.21,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,63.0,57.0,4.35,4.38,2.75,505,0.32,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,65.5,55.0,5.62,5.53,3.65,2686,0.71,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,63.8,56.0,4.68,4.72,3.00,738,0.41,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,60.5,59.0,6.55,6.51,3.95,4882,1.02,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40450,62.7,57.0,7.10,7.04,4.43,10070,1.34,0,0,0,...,0,0,0,0,0,0,0,0,0,0
40451,57.1,60.0,8.31,8.25,4.73,12615,2.02,0,0,0,...,0,0,0,0,1,0,0,0,0,0
40452,62.7,56.0,6.37,6.42,4.01,5457,1.01,0,0,1,...,1,0,0,0,0,0,0,0,0,0
40453,61.9,54.3,4.45,4.47,2.76,456,0.33,0,0,0,...,1,0,0,0,0,0,0,0,0,0


VAMOS A APLICAR EL STANDARD SCALER

In [71]:
scaler = StandardScaler()
X = scaler.fit_transform(diamonds_dummies_cleaned[features])
y = diamonds_dummies_cleaned["price"]

CREAMOS EL MODELO

In [72]:
 X_train, X_validacion, y_train, y_validacion = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
print(f"X_train: {X_train.shape}, X_test: {X_validacion.shape}, y_train: {y_train.shape}")

X_train: (32340, 39), X_test: (8086, 39), y_train: (32340,)


In [74]:
#MODEL

#regressor = LinearRegression()
regressor = RandomForestRegressor(random_state= 42)
hyperparameters = regressor.get_params()
regressor.fit(X_train, y_train)
y_pred_train = regressor.predict(X_validacion)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')
print('Ground truth target:', y_validacion, '\n')
print('Predicted target:', y_pred_train, '\n')

Model: RandomForestRegressor(random_state=42) 

Model hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False} 

Ground truth target: 15580     645
20731     720
24953     680
19363     650
27016     571
         ... 
21012     596
23693    1709
17946    8316
14722    9667
9428     3634
Name: price, Length: 8086, dtype: int64 

Predicted target: [ 643.84  672.5   672.87 ... 8929.23 8758.9  4060.65] 



MIRAMOS EL RMSE(ERROR) QUE TENEMOS EN NUESTRO MODELO

In [75]:
RMSE_ERROR_sklearn = mean_squared_error(y_validacion,y_pred_train, squared = False)
print("TENEMOS UN ERROR DEL " + str(RMSE_ERROR_sklearn) + "%")

TENEMOS UN ERROR DEL 558.0688863000926%


LEEMOS EL DATASET DE TEST 

In [76]:
diamonds_test = pd.read_csv("../KAGGLE_PRUBAS_DATA/DATA/diamonds_test.csv")

CREAMOS LAS VARIABLES DUMMIES DE NUESTRO MODELO YA QUE NO PODEMOS APLICAR LA FUNCION PARA ELIMINAR OUTLIERS PORQUE LO QUE QUEREMOS ES QUE CON EL MODELO QUE HEMOS ENTRENADO NOS HAGA UNA PREDICCION

In [77]:
dummy_clarity_test = pd.get_dummies(diamonds_test["clarity"], prefix= "clarity")
dummy_color_test = pd.get_dummies(diamonds_test["color"], prefix= "color")
dummy_cut_test = pd.get_dummies(diamonds_test["cut"], prefix= "cut")
dummy_city_test = pd.get_dummies(diamonds_test["city"], prefix= "city")


diamonds_dummies_test = pd.concat([diamonds_test, dummy_clarity_test,dummy_color_test,dummy_cut_test,dummy_city_test], axis=1)
lista_columnas_test = diamonds_dummies_test.columns.values.tolist()

ELIMINAMOS LAS COLUMNAS DE LAS QUE HEMOS SACADO LAS DUMMIES

In [78]:
diamonds_dummies_test = diamonds_dummies_test.drop(['clarity','color','cut','city'], axis = 1)

In [79]:
diamonds_dummies_test.head()

,id,carat,depth,table,x,y,z,clarity_I1,clarity_IF,clarity_SI1,...,city_Kimberly,city_Las Vegas,city_London,city_Luxembourg,city_Madrid,city_New York City,city_Paris,city_Surat,city_Tel Aviv,city_Zurich
0,0,0.79,62.7,60.0,5.82,5.89,3.67,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1.20,61.0,57.0,6.81,6.89,4.18,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2,1.57,62.2,61.0,7.38,7.32,4.57,0,0,1,...,1,0,0,0,0,0,0,0,0,0
3,3,0.90,63.8,54.0,6.09,6.13,3.90,0,0,1,...,1,0,0,0,0,0,0,0,0,0
4,4,0.50,62.9,58.0,5.05,5.09,3.19,0,0,0,...,0,0,0,0,0,0,0,0,0,0


APLICAMOS AL TEST EL STANDARD SCALER

In [80]:
scaler = StandardScaler()
diamonds_dummies_test2 = scaler.fit_transform(diamonds_dummies_test[features])

REALIZAMOS LA PREDICTION

In [81]:
y_pred = regressor.predict(diamonds_dummies_test2).clip(0,30000)

print('Model:', regressor, '\n')
print('Model hyperparameters:', hyperparameters, '\n')

Model: RandomForestRegressor(random_state=42) 

Model hyperparameters: {'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False} 



CREAMOS LA SUBMISSION

In [82]:
submission = pd.DataFrame({'id': diamonds_dummies_test["id"] , 'price': y_pred})
submission.shape

(13485, 2)

In [83]:
y_pred.max()

18176.1

In [85]:
submission.to_csv("../KAGGLE_PRUBAS_DATA/DATA/submission2.csv", index=False)